Name: Tan Yee Yang
<br>
Student ID: 20414203
<br>
T must be > 89% 
<br>
use the evaluate algorithm provided and not own algorithm

In [1]:
import os
import numpy as np
import cv2
from glob import glob # use to extract image and mask part from respective folder
from tqdm import tqdm # for progress bar
import imageio # to read scientific picture as it support a lot of image format
from albumentations import HorizontalFlip, VerticalFlip, Rotate # for Data Argumentation
import random
import matplotlib.pyplot as plt
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [2]:
# Parse data
def load_data(path, test_retina_folder_names, train_retina_folder_names, masked_retina_folder_name):

    ## Test image ##
    test_x = []
    test_y = []

    for folder_name in test_retina_folder_names:
        retina_images = sorted(glob(os.path.join(path, folder_name,'*.tif')))
        test_x.extend(retina_images)   # extend the existing list with new retina images
    
    # test_masked_retina_images = sorted(glob(os.path.join(path, masked_retina_folder_name,'*.tif')))
    # test_y = test_masked_retina_images

    
    ## Train image ##
    train_x = [] # for retina images
    train_y = [] # for masked retina images
    
    for folder_name in train_retina_folder_names:
        retina_images = sorted(glob(os.path.join(path, folder_name,'*.tif')))
        train_x.extend(retina_images)   # extend the existing list with new retina images
    
    masked_retina_images = sorted(glob(os.path.join(path, masked_retina_folder_name,'*.tif')))
    
    # Extract base filenames without extension for matching
    base_filenames_train_x = [os.path.splitext(os.path.basename(x))[0] for x in train_x]
    base_filenames_test_x = [os.path.splitext(os.path.basename(x))[0] for x in test_x]
    
    # Filter masked_retina_images based on matching base filenames
    for masked_image in masked_retina_images:
        base_filename_masked = os.path.splitext(os.path.basename(masked_image))[0]
        if base_filename_masked in base_filenames_train_x:
            train_y.append(masked_image)
        if base_filename_masked in base_filenames_test_x:
            test_y.append(masked_image)
        
    # train_masked_retina_images = sorted(glob(os.path.join(path, masked_retina_folder_name,'*.gif')))
    # train_y = train_masked_retina_images
    

    # print (train_x)
    return (train_x, train_y), (test_x, test_y)

In [3]:
# Create a directory
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
def data_argumentation(images, masks, save_path, augment=True):
    # size = (512, 512)
    target_size = (1024, 1024)

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        print(x)
        print(y)
        # make sure image and mask has same name
        # extract the name
        name = x.split('\\')[-1].split('.')[0]
        print(name)

        # read the image and the mask
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0] # mask will not show channel, it is a gray scale image which only has one channel
        print(x.shape, y.shape)

        if augment == True:
            aug = HorizontalFlip(p=1.0) # probability set to 100%
            augmented = aug(image=x, mask=y)
            x1 = augmented['image']
            y1 = augmented['mask']
            
            aug = VerticalFlip(p=1.0) # probability set to 100%
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = Rotate(limit=45, p=1.0) # probability set to 100%, rotate 45 degree
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            X = [x, x1, x2, x3]
            Y = [y, y1, y2 ,y3]
            
        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            # # resize
            # i = cv2.resize(i, size)
            # m = cv2.resize(m, size)

            # Padding
            i = pad_to_target(i, target_size)
            m = pad_to_target(m, target_size, is_mask=True)
            
            tmp_image_name = f'{name}_{index}.png'
            tmp_mask_name = f'{name}_{index}.png'

            image_path = os.path.join(save_path, 'image', tmp_image_name)
            mask_path = os.path.join(save_path, 'mask', tmp_mask_name)
            
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

def pad_to_target(img, target_size, is_mask=False):
    height, width = img.shape[:2]
    # Calculate the padding needed on each side to reach the target size
    delta_w = target_size[1] - width
    delta_h = target_size[0] - height
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    # Use black for padding, which is common in medical imaging
    color = 0 if is_mask else [0, 0, 0]  # Black padding
    # Apply the calculated padding to each side of the image or mask
    padded_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return padded_img

In [5]:
if __name__ == "__main__":
    # Seed
    random.seed(0)

    # Load data
    data_path = './'
    test_retina_image_folders = ['retina_images_01_10', 'retina_images_11_20']
    train_retina_image_folders = ['retina_images_21_30', 'retina_images_31_40']
    masked_retina_image_folder = 'label_images'
    # masked_retina_image_folder = 'mask_images'
    
    (train_x, train_y), (test_x, test_y) = load_data(data_path, test_retina_image_folders, train_retina_image_folders, masked_retina_image_folder)

    # Print the length of the data
    print(f'Train: {len(train_x)} - {len(train_y)}')
    print(f'Test: {len(test_x)} - {len(test_y)}')
    # print(test_y)

    # Create directory to save image from data argumentation
    create_dir('new_data/train/image/')
    create_dir('new_data/train/mask/')
    create_dir('new_data/test/image/')
    create_dir('new_data/test/mask/')

    # Data Argumentation
    data_argumentation(train_x, train_y, 'new_data/train/', augment=True)
    data_argumentation(test_x, test_y, 'new_data/test/', augment=False)

Train: 20 - 20
Test: 20 - 20


  5%|████▏                                                                              | 1/20 [00:00<00:03,  5.75it/s]

./retina_images_21_30\21.tif
./label_images\21.tif
21
(584, 565, 3) (584, 565)
./retina_images_21_30\22.tif
./label_images\22.tif
22
(584, 565, 3) (584, 565)


 15%|████████████▍                                                                      | 3/20 [00:00<00:02,  8.11it/s]

./retina_images_21_30\23.tif
./label_images\23.tif
23
(584, 565, 3) (584, 565)
./retina_images_21_30\24.tif
./label_images\24.tif
24
(584, 565, 3) (584, 565)


...

 60%|█████████████████████████████████████████████████▏                                | 12/20 [00:00<00:00, 24.06it/s]

./retina_images_11_20\11.tif
./label_images\19.tif
11
(584, 565, 3) (584, 565)
./retina_images_11_20\12.tif
./label_images\2.tif
12
(584, 565, 3) (584, 565)
./retina_images_11_20\13.tif
./label_images\20.tif
13
(584, 565, 3) (584, 565)
./retina_images_11_20\14.tif
./label_images\3.tif
14
(584, 565, 3) (584, 565)
./retina_images_11_20\15.tif
./label_images\4.tif
15
(584, 565, 3) (584, 565)


 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [00:00<00:00, 24.21it/s]

./retina_images_11_20\16.tif
./label_images\5.tif
16
(584, 565, 3) (584, 565)
./retina_images_11_20\17.tif
./label_images\6.tif
17
(584, 565, 3) (584, 565)
./retina_images_11_20\18.tif
./label_images\7.tif
18
(584, 565, 3) (584, 565)
./retina_images_11_20\19.tif
./label_images\8.tif
19
(584, 565, 3) (584, 565)
./retina_images_11_20\20.tif
./label_images\9.tif
20
(584, 565, 3) (584, 565)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 24.19it/s]


In [18]:
class conv_block(nn.Module):
    # in_c : number of input channel
    # out_c : number of output channel
    def __init__(self, in_c, out_c):
        super().__init__()

        # 3 x 3 convolution layer
        # Padding in CNN refers to the addition of extra pixels around the borders of the input images or feature map
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)
        
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)
        
        # ReLU activation function
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        # print(x.shape)
    
        # input of second layer is output from first layar
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        return x

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        """
        Pooling Layers, also known as downsample layers, are an essential component of convolutional neural networks 
        (CNNs) used in deep learning. 
        They are responsible for reducing the spatial dimensions of the input data, in terms of width and height, 
        while retaining the most important information.
        """
        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2)) # 2 x 2 max pooling

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)

        """
        Example:
        torch.Size([2, 64, 128, 128])
        torch.Size([2, 64, 128, 128]) torch.Size([2, 64, 64, 64])
        """
        return x, p # x is output of convolution block and p is output of pooling block

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        # 2 x 2 transpose convolution, we want to increase / upsample so:
        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c + out_c, out_c)
        
    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1) # axis one contain the number of channel
        x = self.conv(x) 

        return(x)
        
class build_unet(nn.Module):
    def __init__(self):
        super().__init__()

        ## Encoder ##
        self.e1 = encoder_block(3, 64)     # 3 because rgb got 3 channel
        self.e2 = encoder_block(64, 128)   # 3 because rgb got 3 channel
        self.e3 = encoder_block(128, 256)  # 3 because rgb got 3 channel
        self.e4 = encoder_block(256, 512)  # 3 because rgb got 3 channel
        self.e5 = encoder_block(512, 1024) # 3 because rgb got 3 channel

        ## Bottleneck ##
        self.b = conv_block(1024, 2048) # Bridge layer with just conv_block

        ## Decoder ##
        self.d1 = decoder_block(2048, 1024) # the feature highend increases by 2 output feature map decreasre by 2
        self.d2 = decoder_block(1024, 512) # the feature highend increases by 2 output feature map decreasre by 2
        self.d3 = decoder_block(512, 256)  # the feature highend increases by 2 output feature map decreasre by 2
        self.d4 = decoder_block(256, 128)  # the feature highend increases by 2 output feature map decreasre by 2
        self.d5 = decoder_block(128, 64)   # the feature highend increases by 2 output feature map decreasre by 2
        
        ## Classifier ##
        # To generate the segmentation map
        # 1 because mask is one channel color only
        self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)
    
    def forward(self, inputs):
        ## Encoder ##
        # Act as skip connection for the decoder
        s1, p1 = self.e1(inputs) # s1 skip connection p1 pooling output
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)
        s5, p5 = self.e5(p4)

        ## Bottleneck ## 
        b = self.b(p5)
        # print(s1.shape, s2.shape, s3.shape, s4.shape)
        """
        torch.Size([2, 64, 512, 512])  - 512 x 512 same as input size 
        torch.Size([2, 128, 256, 256]) - half of input size
        torch.Size([2, 256, 128, 128]) 
        torch.Size([2, 512, 64, 64])
        torch.Size([2, 1024, 32, 32])  - at bridge is 32
        """
        # print(b.shape)

        ## Decoder ##
        d1 = self.d1(b, s5)
        d2 = self.d2(d1, s4)
        d3 = self.d3(d2, s3)
        d4 = self.d4(d3, s2)
        d5 = self.d5(d4, s1)

        outputs = self.outputs(d5)

        # print(f'd4.shape:{d4.shape}')
        
        return outputs
        
if __name__ == "__main__":
    # define batch size, number of channel, batch, width
    x = torch.randn((2, 3, 1024, 1024))
    f = build_unet()
    y = f(x)
    # print(y.shape)


In [11]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

In [7]:
""" Calculate the time taken """
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [8]:
class RetinaDataset(Dataset):
    def __init__(self, images_path, masks_path):

        self.images_path = images_path
        self.masks_path = masks_path
        self.n_samples = len(images_path)
        
    def __getitem__(self, index):
        ## Read image ##
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = image / 255.0 # Normalize the image , cv2 image is channel last, (512, 512 ,3)
        image = np.transpose(image, (2, 0, 1)) # pytorch is channel first approach, (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image) # create a tensor from a NumPy array

        ## Read mask ##
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = mask / 255.0 # (512, 512)
        mask = np.expand_dims(mask, axis = 0) # np.expand_dims will expand the dimension by 1, axis = 0 is expand the dimension at the front, expand to first axis, (1, 512, 512)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)

        return image, mask
        
    def __len__(self):
        """
        Return the number of sample
        """
        return self.n_samples

Training function

In [9]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0

    model.train()
    for x, y in loader: # x and y are tensors
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y) # y is ground truth value
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss / len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y in loader: # x and y are tensors
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

        epoch_loss = epoch_loss / len(loader)
    return epoch_loss

Executing the training code

In [19]:
if __name__ == "__main__":
    random.seed(0)

    ## Directory ##
    create_dir('files')

    ## Load dataset ##
    train_x = sorted(glob('./new_data/train/image/*'))
    train_y = sorted(glob('./new_data/train/mask/*'))
    
    valid_x = sorted(glob('./new_data/test/image/*'))
    valid_y = sorted(glob('./new_data/test/mask/*'))

    data_str = f'Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}'
    print(data_str)

    ## Hyperparameters ##
    # H = 512
    # W = 512
    H = 1024
    W = 1024
    size = (H, W)
    batch_size = 2
    num_epochs = 50
    lr = 1e-4 # learning rate
    checkpoint_path = 'files/checkpoint.pth'

    ## Dataset and Loader ##
    train_dataset = RetinaDataset(train_x, train_y)
    valid_dataset = RetinaDataset(valid_x, valid_y)

    train_loader = DataLoader(
        dataset = train_dataset,
        batch_size = batch_size, # sample per batch to load
        shuffle = True,
        num_workers = 0 # num of subprocess
    )

    valid_loader = DataLoader(
        dataset = valid_dataset,
        batch_size = batch_size,
        shuffle = False,
        num_workers = 0
    )

    device = torch.device('cuda') # GTX2060 6GB
    model = build_unet()
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
    loss_fn = DiceBCELoss() # Dice + BCE Loss

    ## Training the model ##
    best_valid_loss = float('inf')
    
    for epoch in tqdm(range(num_epochs)):
        start_time = time.time()

        train_loss = train(model, train_loader, optimizer, loss_fn, device)
        valid_loss = evaluate(model, valid_loader, loss_fn, device)

        ## Save the model ##
        if valid_loss < best_valid_loss:
            data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
            print(data_str)

            best_valid_loss = valid_loss
            torch.save(model.state_dict(), checkpoint_path)            
            
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
        data_str += f'\tTrain Loss: {train_loss:.3f}\n'
        data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
        print(data_str)

Dataset Size:
Train: 80 - Valid: 20


  0%|                                                                                           | 0/50 [00:13<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 11.71 GiB is allocated by PyTorch, and 686.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [27]:
def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)
    return mask
    
if __name__ == "__main__":
    ## Seeding ##
    random.seed(0)

    ## Create folder ##
    create_dir('results')
    
    ## load dataset ##
    test_x = sorted(glob("./new_data/test/image/*"))
    test_y = sorted(glob("./new_data/test/mask/*"))

    # print(test_x)
        
    ## Hyperparameters ##
    # W = 512
    # H = 512
    W = 1024
    H = 1024
    size = (W, H)
    checkpoint_path = 'files/checkpoint.pth'
    
    ## Load checkpoint ##
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = build_unet()
    model = model.to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()

    time_taken = []
        
    for i, x in tqdm(enumerate(test_x), total=len(test_x)):
        ## Extract the name ##
        name = x.split('\\')[-1].split('.')[0]
        # print(name)
        
        ## Reading image ##
        image = cv2.imread(x, cv2.IMREAD_COLOR) # (512, 512, 3)
        x = np.transpose(image, (2, 0, 1)) # (3, 512, 512)
        x = x / 255.0
        x = np.expand_dims(x, axis=0) # (1, 3, 512, 512)
        x = x.astype(np.float32)
        x = torch.from_numpy(x)
        x = x.to(device)

        with torch.no_grad(): # disables gradient calculation (We dont want this during inference)
            ## Inference ##
            start_time = time.time()
            pred_y = model(x)
            pred_y = torch.sigmoid(pred_y)
            total_time = time.time() - start_time
            time_taken.append(total_time)

            pred_y = pred_y[0].cpu().numpy() # (1, 512, 512)
            pred_y = np.squeeze(pred_y, axis=0) # (512, 512)

            # Thresholding the predicted mask
            pred_y = pred_y > 0.5
            pred_y = np.array(pred_y, dtype=np.uint8) # data type unsigned int

        ## Save mask ##
        pred_y = mask_parse(pred_y)

        # Upscale the mask to 565x584
        pred_y_resized = cv2.resize(pred_y, (565, 584), interpolation=cv2.INTER_NEAREST)
    
        # Save mask in TIFF format
        cv2.imwrite(f'results/{name}.tif', pred_y_resized * 255)
        # cv2.imwrite(f'results/{name}.png', pred_y * 255)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.27it/s]


In [3]:
import PIL

# img_path_1 = r'C:\Users\11504\Desktop\iip\cw\retina_images_01_10'
# img_path_2 = r'C:\Users\11504\Desktop\iip\cw\retina_images_11_20'
# mask_path = r'C:\Users\11504\Desktop\iip\cw\mask_images'
# label_path = r'C:\Users\11504\Desktop\iip\cw\label_images'
img_path_1 = r'.\retina_images_01_10'
img_path_2 = r'.\retina_images_11_20'
mask_path = r'.\mask_images'
label_path = r'.\label_images'
results_path = './results'

P_total, N_total, T_total = 0., 0., 0.

for i in range(1, 21):
    # Load input image
    # if i < 11:
    #     img = cv2.imread(os.path.join(img_path_1,str(i)+'.tif'),cv2.IMREAD_COLOR)
    # else:
    #     img = cv2.imread(os.path.join(img_path_2,str(i)+'.tif'),cv2.IMREAD_COLOR)
        
    # # Segment the retina blood vessel
    # imgGreen = img[:,:,1]
    # blur1 = cv2.GaussianBlur(imgGreen,(9,9),0)
    # blur2 = cv2.GaussianBlur(imgGreen,(11,11),0)
    # DoG = blur1-blur2;

    # Load and Preprocess the mask
    mask = np.array(PIL.Image.open(os.path.join(mask_path,str(i)+'.tif')))
    # mask_DoG = np.multiply(DoG,mask/255)
    # thres_DoG = mask_DoG>220
    # unique_values = np.unique(mask)
    # print("Unique values in the mask:", unique_values)
    
    thres_DoG = cv2.imread(os.path.join(results_path, str(i) + '_0.tif'), cv2.IMREAD_GRAYSCALE)
    thres_DoG = (thres_DoG == 255).astype(np.uint8)
    print(np.unique(thres_DoG))
    
    # Load the label
    label = np.array(PIL.Image.open(os.path.join(label_path,str(i)+'.tif')))
    
    TP = sum(sum((label==255) & (thres_DoG==1) & (mask == 255)))
    TN = sum(sum((label==0) & (thres_DoG==0) & (mask == 255)))
    FP = sum(sum((label==0) & (thres_DoG==1) & (mask == 255)))
    FN = sum(sum((label==255) & (thres_DoG==0) & (mask == 255)))
    P_total += TP/(TP+FN)
    N_total += TN/(TN+FP)
    T_total += (TP+TN)/(TP+FN+TN+FP)

print('P Average: ', P_total/20)
print('N Average: ', N_total/20)
print('T Average: ', T_total/20)

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
P Average:  0.7751080475420754
N Average:  0.9638021258694751
T Average:  0.9393800747145725
